# Subtract background

Substract a specified value from each image

## Requirements
- A folder with images 

## Config

### The following code imports and declares functions used for the processing:

In [ ]:
#################################
#  Don't modify the code below  #
#################################

import os
import re
import time
import numpy as np
import intake_io
import pandas as pd
from tqdm import tqdm
from am_utils.utils import walk_dir

## Specify data paths and analysis parameters

### Please provide data paths:

`input_dir`: folder with images of cells to be processed

`output_dir`: folder to save results

`bg_value`: number or a list (for each channel)

In [ ]:
input_dir = "/research/sharedresources/cbi/common/Anna/test/input"
output_dir = "/research/sharedresources/cbi/common/Anna/test/bg_corrected"
bg_value = [50, 80, 60]

### The following code lists all image files in the input directory:

In [ ]:
#################################
#  Don't modify the code below  #
#################################

samples = walk_dir(input_dir)

print(f'{len(samples)} images were found:')
print(np.array(samples))

### The following code subtracts background for all images

In [ ]:
#################################
#  Don't modify the code below  #
#################################

bg_value = np.array([bg_value]).ravel()

for sample in tqdm(samples):
    dataset = intake_io.imload(sample)
    channels = dataset.coords['c'].data
    
    if len(bg_value) >= len(channels):
        for i in range(len(channels)):
            img = dataset.loc[dict(c=channels[i])]['image'].data
            img = np.clip(img, bg_value[i], None)
            dataset['image'].loc[dict(c=channels[i])] = img - bg_value[i]
    else:
        img = dataset['image'].data
        img = np.clip(img, bg_value[0], None)
        dataset['image'].data = img - bg_value[0]

    fn_out = sample.replace(input_dir, output_dir)
    os.makedirs(os.path.dirname(fn_out), exist_ok=True)
    intake_io.imsave(dataset, fn_out)